# IoS-001 QUARANTINED Asset Backfill

**Ingen Google Drive - lagrer lokalt i Colab**

Henter prisdata for 45 QUARANTINED assets.
Last ned CSV-filer manuelt på slutten.

In [ ]:
# 1. Setup
!pip install yfinance pandas tqdm -q
!mkdir -p /content/ios001_data
print("Setup OK")

In [ ]:
# 2. Imports og config
import time
from datetime import datetime, timedelta, date
from pathlib import Path
import pandas as pd
import yfinance as yf
from tqdm import tqdm

DATA_DIR = Path("/content/ios001_data")
DELAY_BETWEEN = 8.0
BATCH_PAUSE = 180.0
BATCH_SIZE = 5

In [ ]:
# 3. QUARANTINED assets (45 stk)
ASSETS = {
    # ETFs
    "DIA": "DIA", "IWM": "IWM", "VOO": "VOO", "VTI": "VTI",
    "XLB": "XLB", "XLC": "XLC", "XLE": "XLE", "XLF": "XLF",
    "XLI": "XLI", "XLK": "XLK", "XLP": "XLP", "XLRE": "XLRE",
    "XLU": "XLU", "XLV": "XLV", "XLY": "XLY",
    # US Equities
    "AMZN": "AMZN", "ANSS": "ANSS", "ASML": "ASML",
    "GOOG": "GOOG", "GOOGL": "GOOGL", "META": "META",
    "PARA": "PARA", "SPLK": "SPLK", "TSLA": "TSLA",
    "BRK.B": "BRK-B", "HES": "HES", "JNJ": "JNJ",
    "MA": "MA", "SQ": "SQ", "UNH": "UNH", "V": "V",
    # Oslo
    "BELCO.OL": "BELCO.OL", "CRAYN.OL": "CRAYN.OL",
    "FLNG.OL": "FLNG.OL", "GJFAH.OL": "GJFAH.OL",
    "GOGL.OL": "GOGL.OL", "KAHOT.OL": "KAHOT.OL",
    "PROTCT.OL": "PROTCT.OL", "REC.OL": "REC.OL",
    "SCHA.OL": "SCHA.OL", "SCHB.OL": "SCHB.OL",
    "SRBNK.OL": "SRBNK.OL", "XXL.OL": "XXL.OL",
    # Other
    "BT.A.L": "BT-A.L", "STM.PA": "STM.PA"
}
print(f"{len(ASSETS)} assets")

In [ ]:
# 4. Fetch funksjon
def fetch(ticker, start, end, retries=5):
    for i in range(retries):
        try:
            if i > 0:
                time.sleep(30 * (2 ** (i-1)))
            df = yf.download(ticker, start=start, end=end,
                           auto_adjust=False, progress=False, threads=False)
            if df is not None and not df.empty:
                return df.dropna(subset=['Close'])
        except Exception as e:
            if '429' in str(e) or 'rate' in str(e).lower():
                print(f"  Rate limit, venter 10 min...")
                time.sleep(600)
    return None

In [ ]:
# 5. KJØR BACKFILL
end = (date.today() - timedelta(days=1)).strftime('%Y-%m-%d')
start = (date.today() - timedelta(days=3650)).strftime('%Y-%m-%d')

results = {"ok": [], "fail": []}
batch = 0

for i, (cid, ticker) in enumerate(tqdm(ASSETS.items())):
    print(f"\n[{i+1}/{len(ASSETS)}] {cid}...")
    
    df = fetch(ticker, start, end)
    
    if df is None or df.empty:
        print(f"  FEIL")
        results["fail"].append(cid)
    else:
        # Lagre CSV
        out = pd.DataFrame({
            'listing_id': cid,
            'date': df.index,
            'open': df['Open'], 'high': df['High'],
            'low': df['Low'], 'close': df['Close'],
            'adj_close': df.get('Adj Close', df['Close']),
            'volume': df['Volume'].astype('Int64'),
            'price_type': 'RAW', 'resolution': '1d',
            'data_source': 'yfinance', 'adr_epoch': 'COLAB'
        })
        path = DATA_DIR / f"{cid.replace('.', '_')}.csv"
        out.to_csv(path, index=False)
        print(f"  OK: {len(df)} rader")
        results["ok"].append(cid)
    
    batch += 1
    time.sleep(DELAY_BETWEEN)
    if batch >= BATCH_SIZE:
        batch = 0
        print(f"\nPause {BATCH_PAUSE}s...")
        time.sleep(BATCH_PAUSE)

print(f"\n\nFERDIG: {len(results['ok'])} OK, {len(results['fail'])} feilet")

In [ ]:
# 6. Last ned som ZIP
import shutil
shutil.make_archive('/content/ios001_data', 'zip', '/content/ios001_data')

from google.colab import files
files.download('/content/ios001_data.zip')
print("Last ned ios001_data.zip og pakk ut lokalt")

In [ ]:
# 7. Vis feilede (for manuell retry)
if results.get('fail'):
    print("Feilede assets:")
    for cid in results['fail']:
        print(f"  {cid}")